In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1' # only for Intel CPU
import time
import pickle
import numpy as np
import multiprocess as mp
from multiprocess.managers import SyncManager
import tensorflow as tf
from libs.FourInARow import FourInARow
from libs.model import create_az_model, CachedModel
from libs.Trainer import Trainer
from libs.MCTS import MCTS

In [2]:
n_cores = mp.cpu_count()
print(f"[Number of cores]: {n_cores}")

[Number of cores]: 16


# Test FourInARow engine

coordinates

In [3]:
np.arange(42).reshape(6,7)

array([[ 0,  1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12, 13],
       [14, 15, 16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25, 26, 27],
       [28, 29, 30, 31, 32, 33, 34],
       [35, 36, 37, 38, 39, 40, 41]])

In [4]:
fiar = FourInARow()

assert the action size

In [5]:
assert fiar.get_action_size()==7

assert the available actions

In [6]:
assert (fiar.get_available_actions(np.zeros(42))==np.ones(7)).all()
assert (fiar.get_reward_for_player(np.zeros(42),1)==None)
assert (fiar.get_reward_for_player(np.zeros(42),-1)==None)
fiar.play(np.zeros(42))
assert (fiar.check_if_winner(1)==False )
assert (fiar.check_if_winner(-1)==False )



assert (fiar.get_available_actions(np.ones(42))==np.zeros(7)).all()
assert (fiar.get_reward_for_player(np.ones(42),1)==1)
assert (fiar.get_reward_for_player(np.ones(42),-1)==-1)
fiar.play(-1*np.ones(42))
assert (fiar.check_if_winner(1)==False )
assert (fiar.check_if_winner(-1)==True )

In [7]:
case = np.zeros(42)
case[-7:] = 1

columns_to_fill = [0,3]
for c in columns_to_fill:
    case[c::7] = 1


solution = np.ones(7)
solution[columns_to_fill] = 0

print(case.reshape(6,7))

[[1. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1.]]


In [8]:
assert (fiar.get_available_actions(case)==solution).all()

fiar.play(case)

assert (fiar.check_if_winner(1)==True )
assert (fiar.check_if_winner(-1)==False )
assert (fiar.get_reward_for_player(np.ones(42),1)==1)
assert (fiar.get_reward_for_player(np.ones(42),-1)==-1)

assert (fiar.get_board_from_player(player=1)==case).all()
assert (fiar.get_board_from_player(player=-1)==-case).all()

fiar.play(case*-1)

assert (fiar.check_if_winner(1)==False )
assert (fiar.check_if_winner(-1)==True )
assert (fiar.get_reward_for_player(-np.ones(42),1)==-1)
assert (fiar.get_reward_for_player(-np.ones(42),-1)==1)

assert (fiar.get_board_from_player(player=1)==-case).all()
assert (fiar.get_board_from_player(player=-1)==case).all()

In [9]:
next_player = 1
next_state, next_player = fiar.next_state(board=np.zeros(42), player=next_player, action=0)
print(next_state.reshape(6,7))

next_state, next_player = fiar.next_state(board=next_state, player=next_player, action=0)
print(next_state.reshape(6,7))

next_state, next_player = fiar.next_state(board=next_state, player=next_player, action=3)
print(next_state.reshape(6,7))

next_state, next_player = fiar.next_state(board=next_state, player=next_player, action=2)
print(next_state.reshape(6,7))

next_state, next_player = fiar.next_state(board=next_state, player=next_player, action=5)
print(next_state.reshape(6,7))

next_state, next_player = fiar.next_state(board=next_state, player=next_player, action=6)
print(next_state.reshape(6,7))

print("Last board for player 1")
print(fiar.get_board_from_player(player=1).reshape(6,7))
print("Last board for player -1")
print(fiar.get_board_from_player(player=-1).reshape(6,7))

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0. -1.  1.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0. -1.  1.  0.  1.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 

# Test TF model

In [10]:
model = create_az_model()

In [11]:
inputs = fiar.get_board_from_player(player=-1)[np.newaxis,:]

In [12]:
model(inputs)

(<tf.Tensor: shape=(1, 7), dtype=float32, numpy=
 array([[0.14285715, 0.14285715, 0.14285715, 0.14285715, 0.14285715,
         0.14285715, 0.14285715]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.]], dtype=float32)>)

In [13]:
tf.keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


# MCTS

In [14]:
mcts = MCTS(game=FourInARow(), n_simulations=100)

In [15]:
state = np.zeros(42)

In [16]:
root = mcts.run(model=None, state=state, player=1)

In [17]:
root

State:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Player:1
Value:0.0
Leaf:False
Visits:100
Children:
 -0: prior=0.14285714285714285
 -1: prior=0.14285714285714285
 -2: prior=0.14285714285714285
 -3: prior=0.14285714285714285
 -4: prior=0.14285714285714285
 -5: prior=0.14285714285714285
 -6: prior=0.14285714285714285

In [18]:
root.children[2]

State:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
Player:-1
Value:0.0
Leaf:False
Visits:14
Children:
 -0: prior=0.14285714285714285
 -1: prior=0.14285714285714285
 -2: prior=0.14285714285714285
 -3: prior=0.14285714285714285
 -4: prior=0.14285714285714285
 -5: prior=0.14285714285714285
 -6: prior=0.14285714285714285

In [19]:
root.children[2].children

{0: State:
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.
   0.  1.  0.  0.  0.  0.]
 Player:1
 Value:0.0
 Leaf:False
 Visits:2
 Children:
  -0: prior=0.14285714285714285
  -1: prior=0.14285714285714285
  -2: prior=0.14285714285714285
  -3: prior=0.14285714285714285
  -4: prior=0.14285714285714285
  -5: prior=0.14285714285714285
  -6: prior=0.14285714285714285,
 1: State:
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  -1.  1.  0.  0.  0.  0.]
 Player:1
 Value:0.0
 Leaf:False
 Visits:2
 Children:
  -0: prior=0.14285714285714285
  -1: prior=0.14285714285714285
  -2: prior=0.14285714285714285
  -3: prior=0.14285714285714285
  -4: prior=0.14285714285714285
  -5: prior=0.14285714285714285
  -6: prior=0.14285714285714285,
 2: State:
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  

# Generating the dataset

In [20]:
STARTING_TEMPERATURE = 10.0

In [21]:
start_time = time.time()
trainer = Trainer(game = FourInARow, mcts = MCTS, model=None)
dataset = trainer.create_dataset(number_of_games=2, temperature=STARTING_TEMPERATURE)
print("Running time: %s seconds" % (time.time() - start_time))

Running time: 4.005084276199341 seconds


Parallelizing the code to speed up the generation of games

In [22]:
def parallel_fn(n):
    from libs.Trainer import Trainer
    from libs.FourInARow import FourInARow
    from libs.MCTS import MCTS
    N_GAMES_PER_JOB = 25
    STARTING_TEMPERATURE = 10.0
    trainer = Trainer(game = FourInARow, mcts = MCTS, model=None)
    game_batch = trainer.create_dataset(number_of_games=N_GAMES_PER_JOB, temperature=STARTING_TEMPERATURE)
    return game_batch

In [23]:
#if __name__ == "__main__":
start_time = time.time()
ctx = mp.get_context("spawn")
pool = ctx.Pool(n_cores)
dataset = pool.map(parallel_fn, range(n_cores))
print("Running time: %s seconds" % (time.time() - start_time))

Running time: 139.8901174068451 seconds


In [24]:
flatten_dataset = []
for batch in dataset:
    flatten_dataset += batch

In [25]:
len(flatten_dataset)

7692

# Training

In [27]:
trainer = Trainer(game = FourInARow, mcts = MCTS, model=create_az_model())

In [28]:
fit_history = trainer.train(flatten_dataset, n_epochs=1000, batch_size=30, learning_rate=1e-2)

Epoch 1/1000


231/231 [==============================] - 4s 11ms/step - loss: 4.2212 - policy_loss: 2.0336 - value_loss: 1.9404 - val_loss: 3.8646 - val_policy_loss: 1.9442 - val_value_loss: 1.7371 - lr: 0.0100
Epoch 2/1000
231/231 [==============================] - 2s 10ms/step - loss: 3.9931 - policy_loss: 1.9220 - value_loss: 1.9388 - val_loss: 4.0074 - val_policy_loss: 1.9677 - val_value_loss: 1.9428 - lr: 0.0100
Epoch 3/1000
231/231 [==============================] - 2s 10ms/step - loss: 3.9364 - policy_loss: 1.9055 - value_loss: 1.9399 - val_loss: 3.9755 - val_policy_loss: 1.9396 - val_value_loss: 1.9429 - lr: 0.0100
Epoch 4/1000
231/231 [==============================] - 2s 10ms/step - loss: 3.9348 - policy_loss: 1.8847 - value_loss: 1.9369 - val_loss: 4.7572 - val_policy_loss: 2.4777 - val_value_loss: 2.0571 - lr: 0.0100
Epoch 5/1000
231/231 [==============================] - 2s 10ms/step - loss: 3.2006 - policy_loss: 1.8890 - value_loss: 1.0439 - val_loss: 3.0971 - val_policy

Epoch 38/1000
231/231 [==============================] - 2s 9ms/step - loss: 2.5751 - policy_loss: 1.5201 - value_loss: 0.9991 - val_loss: 2.7538 - val_policy_loss: 1.6992 - val_value_loss: 0.9992 - lr: 2.2823e-04
Epoch 39/1000
231/231 [==============================] - 2s 9ms/step - loss: 2.5735 - policy_loss: 1.5194 - value_loss: 0.9991 - val_loss: 2.7522 - val_policy_loss: 1.6985 - val_value_loss: 0.9992 - lr: 1.7249e-04
Epoch 40/1000
231/231 [==============================] - 2s 9ms/step - loss: 2.5727 - policy_loss: 1.5193 - value_loss: 0.9991 - val_loss: 2.7516 - val_policy_loss: 1.6985 - val_value_loss: 0.9992 - lr: 1.2907e-04
Epoch 41/1000
231/231 [==============================] - 2s 9ms/step - loss: 2.5701 - policy_loss: 1.5173 - value_loss: 0.9991 - val_loss: 2.7508 - val_policy_loss: 1.6981 - val_value_loss: 0.9992 - lr: 1.0000e-04
Epoch 42/1000
231/231 [==============================] - 2s 9ms/step - loss: 2.5699 - policy_loss: 1.5176 - value_loss: 0.9991 - val_loss: 2.751

231/231 [==============================] - 2s 10ms/step - loss: 2.5356 - policy_loss: 1.5067 - value_loss: 0.9991 - val_loss: 2.7267 - val_policy_loss: 1.6979 - val_value_loss: 0.9992 - lr: 1.0000e-04
Epoch 115/1000
231/231 [==============================] - 2s 9ms/step - loss: 2.5345 - policy_loss: 1.5058 - value_loss: 0.9991 - val_loss: 2.7240 - val_policy_loss: 1.6953 - val_value_loss: 0.9992 - lr: 1.0000e-04
Epoch 116/1000
231/231 [==============================] - 2s 9ms/step - loss: 2.5354 - policy_loss: 1.5069 - value_loss: 0.9991 - val_loss: 2.7234 - val_policy_loss: 1.6949 - val_value_loss: 0.9992 - lr: 1.0000e-04
Epoch 117/1000
231/231 [==============================] - 2s 9ms/step - loss: 2.5348 - policy_loss: 1.5065 - value_loss: 0.9991 - val_loss: 2.7249 - val_policy_loss: 1.6966 - val_value_loss: 0.9992 - lr: 1.0000e-04
Epoch 118/1000
231/231 [==============================] - 2s 9ms/step - loss: 2.5346 - policy_loss: 1.5065 - value_loss: 0.9991 - val_loss: 2.7246 - val_p

In [29]:
trainer.save_model('saved_model/tmp_model')

INFO:tensorflow:Assets written to: saved_model/tmp_model\assets


INFO:tensorflow:Assets written to: saved_model/tmp_model\assets


# Generating the self-playing dataset (2nd)

loaded_model = tf.keras.models.load_model("saved_model/alpha_zero_model/")

# speed test on 4 games
start_time = time.time()
trainer = Trainer(game = FourInARow, mcts = MCTS, model=loaded_model)
dataset = trainer.create_dataset(number_of_games=4, temperature=1)
print("Running time: %s seconds" % (time.time() - start_time))

In [30]:
N_ROUNDS = 40

In [33]:
def parallel_fn(job_n, cache, n):
    
    from libs.Trainer import Trainer
    from libs.FourInARow import FourInARow
    from libs.model import CachedModel
    from libs.MCTS import MCTS
    import tensorflow as tf
    N_GAMES_PER_JOB = 25
    STARTING_TEMPERATURE = 10.0
    
    model = tf.keras.models.load_model("saved_model/tmp_model")
    trainer = Trainer(game = FourInARow, mcts = MCTS, model=CachedModel(model=model))
    
    game_batch = trainer.create_dataset(number_of_games=N_GAMES_PER_JOB, temperature=STARTING_TEMPERATURE/n)
    
    return game_batch

In [ ]:
if __name__ == '__main__':

    self_play_dataset = flatten_dataset
    
    model = create_az_model()

    for n in range(N_ROUNDS):

        start_time = time.time()
        ctx = mp.get_context("spawn")

        with SyncManager() as manager:

            shared_cache = manager.dict()
            print(f"shared_cache type: {type(shared_cache)}")
            args = [(job_n, shared_cache, n+1) for job_n in range(n_cores)]
            print(f"Number of args: {len(args)}")
            print(args)

            with ctx.Pool(n_cores) as pool:

                round_dataset = pool.starmap(parallel_fn, args)

        print(f"Running time {n+1}-th round: {time.time() - start_time} seconds")

        flatten_round_dataset = []
        for batch in round_dataset:
            flatten_round_dataset += batch
        self_play_dataset += flatten_round_dataset
        print(f"Dataset size: {len(self_play_dataset)}")

        trainer = Trainer(game = FourInARow, mcts = MCTS, model=model)
        fit_history = trainer.train(self_play_dataset, n_epochs=50, batch_size=50, learning_rate=1e-2)

        trainer.model.save("saved_model/tmp_model")



shared_cache type: <class 'multiprocess.managers.DictProxy'>
Number of args: 16
[(0, <DictProxy object, typeid 'dict' at 0x228764f8670>, 1), (1, <DictProxy object, typeid 'dict' at 0x228764f8670>, 1), (2, <DictProxy object, typeid 'dict' at 0x228764f8670>, 1), (3, <DictProxy object, typeid 'dict' at 0x228764f8670>, 1), (4, <DictProxy object, typeid 'dict' at 0x228764f8670>, 1), (5, <DictProxy object, typeid 'dict' at 0x228764f8670>, 1), (6, <DictProxy object, typeid 'dict' at 0x228764f8670>, 1), (7, <DictProxy object, typeid 'dict' at 0x228764f8670>, 1), (8, <DictProxy object, typeid 'dict' at 0x228764f8670>, 1), (9, <DictProxy object, typeid 'dict' at 0x228764f8670>, 1), (10, <DictProxy object, typeid 'dict' at 0x228764f8670>, 1), (11, <DictProxy object, typeid 'dict' at 0x228764f8670>, 1), (12, <DictProxy object, typeid 'dict' at 0x228764f8670>, 1), (13, <DictProxy object, typeid 'dict' at 0x228764f8670>, 1), (14, <DictProxy object, typeid 'dict' at 0x228764f8670>, 1), (15, <DictProx

Epoch 35/50
243/243 [==============================] - 3s 11ms/step - loss: 2.9400 - policy_loss: 1.9411 - value_loss: 0.9988 - val_loss: 2.9431 - val_policy_loss: 1.9444 - val_value_loss: 0.9987 - lr: 4.9787e-04
Epoch 36/50
243/243 [==============================] - 3s 11ms/step - loss: 2.9400 - policy_loss: 1.9411 - value_loss: 0.9988 - val_loss: 2.9430 - val_policy_loss: 1.9443 - val_value_loss: 0.9987 - lr: 3.8774e-04
Epoch 37/50
243/243 [==============================] - 3s 11ms/step - loss: 2.9400 - policy_loss: 1.9411 - value_loss: 0.9988 - val_loss: 2.9430 - val_policy_loss: 1.9443 - val_value_loss: 0.9987 - lr: 2.9897e-04
Epoch 38/50
243/243 [==============================] - 3s 11ms/step - loss: 2.9399 - policy_loss: 1.9411 - value_loss: 0.9988 - val_loss: 2.9430 - val_policy_loss: 1.9443 - val_value_loss: 0.9987 - lr: 2.2823e-04
Epoch 39/50
243/243 [==============================] - 3s 11ms/step - loss: 2.9399 - policy_loss: 1.9411 - value_loss: 0.9988 - val_loss: 2.9430 - v

INFO:tensorflow:Assets written to: saved_model/tmp_model\assets


shared_cache type: <class 'multiprocess.managers.DictProxy'>
Number of args: 16
[(0, <DictProxy object, typeid 'dict' at 0x22870ec4490>, 2), (1, <DictProxy object, typeid 'dict' at 0x22870ec4490>, 2), (2, <DictProxy object, typeid 'dict' at 0x22870ec4490>, 2), (3, <DictProxy object, typeid 'dict' at 0x22870ec4490>, 2), (4, <DictProxy object, typeid 'dict' at 0x22870ec4490>, 2), (5, <DictProxy object, typeid 'dict' at 0x22870ec4490>, 2), (6, <DictProxy object, typeid 'dict' at 0x22870ec4490>, 2), (7, <DictProxy object, typeid 'dict' at 0x22870ec4490>, 2), (8, <DictProxy object, typeid 'dict' at 0x22870ec4490>, 2), (9, <DictProxy object, typeid 'dict' at 0x22870ec4490>, 2), (10, <DictProxy object, typeid 'dict' at 0x22870ec4490>, 2), (11, <DictProxy object, typeid 'dict' at 0x22870ec4490>, 2), (12, <DictProxy object, typeid 'dict' at 0x22870ec4490>, 2), (13, <DictProxy object, typeid 'dict' at 0x22870ec4490>, 2), (14, <DictProxy object, typeid 'dict' at 0x22870ec4490>, 2), (15, <DictProx

INFO:tensorflow:Assets written to: saved_model/tmp_model\assets


...

shared_cache type: <class 'multiprocess.managers.DictProxy'>
Number of args: 16
[(0, <DictProxy object, typeid 'dict' at 0x22866c57cd0>, 5), (1, <DictProxy object, typeid 'dict' at 0x22866c57cd0>, 5), (2, <DictProxy object, typeid 'dict' at 0x22866c57cd0>, 5), (3, <DictProxy object, typeid 'dict' at 0x22866c57cd0>, 5), (4, <DictProxy object, typeid 'dict' at 0x22866c57cd0>, 5), (5, <DictProxy object, typeid 'dict' at 0x22866c57cd0>, 5), (6, <DictProxy object, typeid 'dict' at 0x22866c57cd0>, 5), (7, <DictProxy object, typeid 'dict' at 0x22866c57cd0>, 5), (8, <DictProxy object, typeid 'dict' at 0x22866c57cd0>, 5), (9, <DictProxy object, typeid 'dict' at 0x22866c57cd0>, 5), (10, <DictProxy object, typeid 'dict' at 0x22866c57cd0>, 5), (11, <DictProxy object, typeid 'dict' at 0x22866c57cd0>, 5), (12, <DictProxy object, typeid 'dict' at 0x22866c57cd0>, 5), (13, <DictProxy object, typeid 'dict' at 0x22866c57cd0>, 5), (14, <DictProxy object, typeid 'dict' at 0x22866c57cd0>, 5), (15, <DictProx

INFO:tensorflow:Assets written to: saved_model/tmp_model\assets


shared_cache type: <class 'multiprocess.managers.DictProxy'>
Number of args: 16
[(0, <DictProxy object, typeid 'dict' at 0x2280c667220>, 6), (1, <DictProxy object, typeid 'dict' at 0x2280c667220>, 6), (2, <DictProxy object, typeid 'dict' at 0x2280c667220>, 6), (3, <DictProxy object, typeid 'dict' at 0x2280c667220>, 6), (4, <DictProxy object, typeid 'dict' at 0x2280c667220>, 6), (5, <DictProxy object, typeid 'dict' at 0x2280c667220>, 6), (6, <DictProxy object, typeid 'dict' at 0x2280c667220>, 6), (7, <DictProxy object, typeid 'dict' at 0x2280c667220>, 6), (8, <DictProxy object, typeid 'dict' at 0x2280c667220>, 6), (9, <DictProxy object, typeid 'dict' at 0x2280c667220>, 6), (10, <DictProxy object, typeid 'dict' at 0x2280c667220>, 6), (11, <DictProxy object, typeid 'dict' at 0x2280c667220>, 6), (12, <DictProxy object, typeid 'dict' at 0x2280c667220>, 6), (13, <DictProxy object, typeid 'dict' at 0x2280c667220>, 6), (14, <DictProxy object, typeid 'dict' at 0x2280c667220>, 6), (15, <DictProx

INFO:tensorflow:Assets written to: saved_model/tmp_model\assets


shared_cache type: <class 'multiprocess.managers.DictProxy'>
Number of args: 16
[(0, <DictProxy object, typeid 'dict' at 0x2287a1fa490>, 7), (1, <DictProxy object, typeid 'dict' at 0x2287a1fa490>, 7), (2, <DictProxy object, typeid 'dict' at 0x2287a1fa490>, 7), (3, <DictProxy object, typeid 'dict' at 0x2287a1fa490>, 7), (4, <DictProxy object, typeid 'dict' at 0x2287a1fa490>, 7), (5, <DictProxy object, typeid 'dict' at 0x2287a1fa490>, 7), (6, <DictProxy object, typeid 'dict' at 0x2287a1fa490>, 7), (7, <DictProxy object, typeid 'dict' at 0x2287a1fa490>, 7), (8, <DictProxy object, typeid 'dict' at 0x2287a1fa490>, 7), (9, <DictProxy object, typeid 'dict' at 0x2287a1fa490>, 7), (10, <DictProxy object, typeid 'dict' at 0x2287a1fa490>, 7), (11, <DictProxy object, typeid 'dict' at 0x2287a1fa490>, 7), (12, <DictProxy object, typeid 'dict' at 0x2287a1fa490>, 7), (13, <DictProxy object, typeid 'dict' at 0x2287a1fa490>, 7), (14, <DictProxy object, typeid 'dict' at 0x2287a1fa490>, 7), (15, <DictProx

https://medium.com/oracledevs/lessons-from-alpha-zero-part-5-performance-optimization-664b38dc509e

https://stackoverflow.com/questions/16589791/most-efficient-property-to-hash-for-numpy-array

https://developer.nvidia.com/tensorrt

https://www.tensorflow.org/model_optimization

https://www.tensorflow.org/model_optimization/guide/combine/collaborative_optimization

In [ ]:
1. averaging
2. CPUTC add c=3 or 4